In [ ]:
import csv 
# from fpdf import FPDF
# !pip install fasttext-langdetect
from ftlangdetect import detect

Converting .csv file to pretty .pdf:  

| REGION_NAME | BUSINESS_AREA_NAME | NarrativeTitle                   | NarrativeText |
| ----------- | ------------------ | -------------------------------- | ------------- |
| EAPR        | Cambodia           | Lessons Learned and Innovations  | sdfhjsdf      |
| EAPR        | China              | Lessons Learned and Innovat ions | aslkdgvj      |  

PDF:

Year - 2022, Country - Combodia   
Lessons learned and Innovations for the country Cambodia in the region EAPR for the year 2022 were as follows:  
sdfkljsdf  

Year - 2022, Country - China  
Lessons learned and Innovations for the country China in the region EAPR for the year 2022 were as follows:  
aslkdgvj


In [ ]:
file_path = './'
file_name = 'RAM3 End-Year Summary Narrative Analysis '
file_years = ['2018','2019','2020','2021','2022']

for file_year in file_years:
    with open(file_path+file_name+file_year+'.csv','r') as file_in:
        
        reader = csv.reader(file_in, delimiter=',')
        next(reader,None) # skip header here.
        
        with open(file_path+file_name+file_year+'.txt','w') as file_out:
            count  = 0
            country_name = None
            for row in reader:
                if row :
                    if row[0] == 'HQ': break
                
                    else:
                        country_name = row[1] 
                        h1 = f'\n\nYear - {file_year}, Country - {country_name}.\n'
                        h2 = f'Lessons learned and Innovations for the country {country_name} in the region EAPR for the year {file_year} were as follows:\n'
                        text = row[3]
                        text = text.replace('\n',' ')
                        is_en = detect(text = text, low_memory = True)
                        if is_en['lang'] =='en' and is_en['score']>=0.85:

                            file_out.write(h1)
                            file_out.write(h2)
                            file_out.write(row[3])

            #             pdf.add_page()
            #             pdf.cell(200, 10, txt = h1, ln = 1, align = 'L')
            #             pdf.cell(200, 10, txt = h2, ln = 1, align = 'L')
            #             pdf.cell(200, 10, txt = text, ln = 1, align = 'L')
            # pdf.output(file_path+file_name+file_year+".pdf")   
                
                
            
        

## TODO:  
* Remove non-eng narratives.
* PDF convertion fails at \u2018 character.
